In [1]:
# Preprocessing text data for machine learning applications
# Bag-of-words approaches and sequence-modeling approaches for text processing
# The Transformer architecture
# Sequence-to-sequence learning

In [2]:
# preparing text data
# standardize text, vectrorise text. Convert to lowercase and remove punctuations.
# tokenization
# convert each token to numeric vector, indexing all tokens present in the data.
# one hot encoding or embedding

In [3]:
# Text standardization
# tokenization

In [4]:
# Vocabulary indexing
# padding

In [5]:
# Preparing the IMDB movie reviews data

In [6]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0   306k      0  0:04:28  0:04:28 --:--:--  368k   0     0   173k      0  0:07:54  0:00:24  0:07:30  188k 202k      0  0:06:44  0:00:34  0:06:10  303k:33  409kM    0     0   247k      0  0:05:31  0:01:11  0:04:20  153k  257k      0  0:05:19  0:01:23  0:03:56  342k  0  0:05:17  0:01:24  0:03:53  369k   0   267k      0  0:05:07  0:01:37  0:03:30  143k270k      0  0:05:03  0:01:40  0:03:23  276k  279k      0  0:04:54  0:01:46  0:03:08  404k0:04:44  0:01:51  0:02:53  496k.4M    0     0   299k      0  0:04:33  0:02:04  0:02:29  568kM    0     0   305k      0  0:04:29  0:02:06  0:02:23  601k 0     0   315k      0  0:04:20  0:03:13  0:01:07  281k4  0:01:06  287k   308k      0  0:04:26  0:03:21  0:01:05  170k0   307k      0  0:04:26  0:03:22  0:01:04  169k04:28  0:03:40  0:00:48  213k378k   0     0   306k    

In [7]:
!rm -r aclImdb/train/unsup

In [8]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [10]:
import os, pathlib, shutil, random

In [12]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    
    for fname in val_files:
        shutil.move(train_dir / category / fname, val_dir / category / fname)

In [13]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [14]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'Well I guess I know the answer to that question. For the MONEY! We have been so bombarded with Cat In The Hat advertising and merchandise that we almost believe there has to be something good about this movie. I admit, I thought the trailers looked bad, but I still had to give it a chance. Well I should have went with my instincts. It was a complete piece Hollywood trash. Once again proving that the average person can be programed into believing anything they say is good, must be good. Aside from the insulting fact that the film is only about 80 minutes long, it obviously started with a moth eaten script. It\'s chock full of failed attempts at senseless humor, and awful pastel sceneries. It jumps all over the universe with no destination nor direction. This is then compounded with, ............................yes I\'ll say it, BAD ACTING! I couldn\'t help but 